# Ecoinvent

In [1]:
import brightway2 as bw
from bw2data import mapping

import copy
import time

import numpy as np
import pandas as pd
from scipy import sparse

In [2]:
bw.projects.set_current('ABC unified')

In [3]:
db = bw.Database('ecoinvent 3.5 cutoff')

In [4]:
lcia_method = ('IPCC 2013', 'climate change', 'GWP 100a')

In [5]:
act = [act for act in db if 'transport' in act['name']
                   and 'regional' in act['name']
                   and 'passenger train' in act['name']
                   and 'CH' in act['location']
][0]
demand = {act:1}

In [6]:
LCA = bw.LCA(demand,lcia_method)
LCA.lci()
LCA.lcia()

#Dimensions
n = len(LCA.activity_dict)
m = len(LCA.biosphere_dict)
k = len(bw.methods)

In [7]:
# one_vec = np.ones(m)
# #run LCIA for all methods
# C = np.zeros([m,k])
# im=0
# for method in bw.methods:
#     print(im)
#     LCA = bw.LCA(demand,method=method)
#     LCA.lci()
#     LCA.lcia()
#     C_temp = LCA.characterization_matrix
#     C_dense = C_temp.todense()
#     C[:,im] = np.matmul(C_dense,one_vec)
#     im += 1

In [8]:
#Traditional LCA
A = LCA.technosphere_matrix
B = LCA.biosphere_matrix
C = LCA.characterization_matrix
f = LCA.demand_array

In [9]:
Cext = sum(C)
k = Cext.shape[0]

#Extended LCA
Cell1 = sparse.vstack([B,np.zeros([k,n])])
Col1  = sparse.vstack([A,Cell1])

Cell2 = sparse.vstack([np.zeros([n,m]),np.diag(-np.ones(m))])
Col2  = sparse.vstack([Cell2,Cext])

Dext  = np.zeros([n,k])
Cell3 = sparse.vstack([Dext,np.zeros([m,k])])
Col3  = sparse.vstack([Cell3,np.diag(-np.ones(k))]) 

Row1  = sparse.hstack([Col1,Col2])
Aext  = sparse.hstack([Row1,Col3])
fext = sparse.hstack([f,np.zeros(m+k)]).transpose()
# #convert to sparse matrix representation
Aext = sparse.csr_matrix(Aext)
fext = sparse.csr_matrix(fext)

In [10]:
t1 = time.time()
g = sparse.linalg.spsolve(A,f)
g = B@g
# g = np.matmul(B.todense(),g.transpose())
# g = np.matmul(C.todense(),g.transpose())
g = C@g
t2 = time.time()
delta_t = t2-t1

t1_ext = time.time()
g_ext = sparse.linalg.spsolve(Aext,fext)[:m]
t2_ext = time.time()
delta_t_ext = t2_ext-t1_ext

In [11]:
delta_t

1.1560571193695068

In [12]:
delta_t_ext

2.800372362136841

In [13]:
jl = 20
g[:jl]

array([0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0., 0.,
       0., 0., 0.])

In [14]:
g_ext[:jl]

array([-3.87270387e-18,  0.00000000e+00,  5.64286341e-08,  8.10340750e-12,
        3.95692084e-22,  0.00000000e+00,  1.48110174e-11,  1.46885287e-07,
        1.41652867e-04,  0.00000000e+00,  0.00000000e+00,  9.93685650e-16,
        4.37294922e-05,  0.00000000e+00,  2.29243970e-06,  1.52624249e-08,
        2.35642452e-28,  1.88615319e-07,  0.00000000e+00,  7.99946222e-08])

In [15]:
error = sum((g-g_ext)**2)/n
error

5.830872984344387e-07

# Results from the paper of Weidema
https://doi.org/10.1007/s11367-017-1398-4

In [ ]:
n_pr = 3 #number of processes
n_el = 2 #number of elementary flows
n_ch = 3 #number of number of characterization factors
A = np.eye(n_pr)
Bt = np.array([[0.0014,0.005,0.0006],[0.01,0.01,0.04]])
Ct = np.array([[0.2,0.076],[1600,610],[33,0]])
ft = np.array([1,0,0])

### Traditional calculation

In [ ]:
g = np.matmul(Ct,np.matmul(Bt,(np.matmul(np.linalg.inv(A),ft))))

### Expanded calculation

In [ ]:
n_mi = 2 #number of additional midpoints
B = np.vstack((Bt,np.zeros([n_ch+n_mi,n_pr])))
C = np.diag(-np.ones(n_el+n_ch+n_mi))

### My interpretation

In [ ]:
At = np.eye(3)
At[1,0] = 0.9
myA1 = np.vstack([A,Bt])
myA2 = np.vstack([np.zeros([3,2]),np.diag(-np.ones(2))])
myA  = np.hstack([myA1,myA2])
myf  = np.hstack([ft,np.zeros(2)])

In [ ]:
emissions_t = np.matmul(Bt,(np.matmul(np.linalg.inv(At),ft)))
myemissions = np.matmul(np.linalg.inv(myA),myf)

In [ ]:
emissions_t

In [ ]:
myemissions

In [ ]:
myA

In [ ]:
def get_majority_element(a, left, right):
    if left == right:
        return -1
    if left + 1 == right:
        return a[left]
    #write your code here
    return -1

In [ ]:
def get_majority_element(a):
    low,high = 0,len(a)-1
    a = sorted(a)
    
    res = get_majority_element_(a,low,high)
    print(res)
    if res==-1:
        return 0
    else:
        return 1

def get_majority_element_(a,low,high):
    mid = (low+high)//2
    print('ff')
    
    if high-low<=1:
        if a[low]==a[high]:
            return a[low]
        else:
            return -1
    else:
        res1 = get_majority_element_(a,low,mid)
        res2 = get_majority_element_(a,mid+1,high)
        if res1==res2 and res1!=-1:
            return res1
        else:
            return -1
    

In [ ]:
a = [2,2,2,1,2,2,2,2]
get_majority_element(a)